In [4]:
import requests
import numpy as np
import os
from bs4 import BeautifulSoup
import re
import pandas as pd
import zipfile
import pickle

In [7]:
# get the list of ids from 9_way_dataset.zip

with zipfile.ZipFile('9_way_dataset.zip', 'r') as zip_file:
    # Assuming there's only one file inside the ZIP
    filename = zip_file.namelist()[0]
    # Open the file inside the ZIP
    with zip_file.open(filename) as f:
        # Load the pickled object
        data_9 = pickle.load(f)

In [40]:
# get the list of ids from 9_way_dataset
ids = []
for i in range(6,9):
    for data in data_9[i]:
        ids.append(data[0])

ids = list(set(ids))
print(ids)
ids = [int(i) for i in ids]
print(ids)

['00225', '114991', '55554', '112601', '271048', '75864', '14344', '04394', '25459', '117297', '05511', '54618', '58117', '58174', '27752', '29930', '341293', '04404', '03300', '53510', '00402', '01024', '02187', '01998', '02021', '03286', '18993', '07754', '111850', '05857', '96311', '08378', '67709', '02190', '08383', '06401', '112339', '00749', '98350', '140313', '05822', '12668', '02197', '95915', '00822', '05829', '08377', '139909', '06085', '302230', '472732', '18994', '03117', '56693', '475107', '56446', '44117', '04104', '53603', '08410', '03990', '113141', '482981', '53055', '451299', '56708', '76818', '111849', '516478', '68732', '14080', '51743', '315072', '472199', '06432', '08399', '51559', '109693', '06427', '01022', '115494', '12049', '04870', '78311', '06421', '01337', '03135', '95917', '13821', '03122', '06405', '172555', '67814', '111845', '98346', '55011', '55456', '108450', '00832', '03280', '56678', '471873', '08406', '08379', '03282', '54984', '01023', '58719', '4

In [53]:
# get pickle file
with open('9_way_pdf_dict.pkl', 'rb') as f:
    ids_9 = pickle.load(f)
    
url_list = []
not_valid_ids = []
for i in ids:
    try:
        url_list.append(ids_9[i])
    except:
        not_valid_ids.append(i)
        continue
print(len(ids))
print(len(url_list))
print(len(not_valid_ids))

882
475
407


In [54]:
ids_9[883537]

KeyError: 883537

In [ ]:
# functions to get page text and parse out the general information sectioin

def get_page_text(url):
    '''Retrive the text from url.'''
    # parse web content into English text
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check for any request errors

        soup = BeautifulSoup(response.content, 'html.parser')
        # Extract all text from the page, excluding script and style tags
        text = soup.get_text(separator='\n', strip=True)

        return text
    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None

def extract_text_between_strings_second(text, start_string, end_string):
    '''
    Extracts the substring between the second occurrence of start_string and end_string in text.
    '''
    # Find the second occurrence of the start_string (the second occurence of "general information" gives us our metadata)
    if text:
        start_index = text.find(start_string, text.find(start_string) + len(start_string))
        if start_index == -1:  # If start_string is not found
            return None

        # Find the occurrence of the end_string
        end_index = text.find(end_string, start_index)
        if end_index == -1:  # If end_string is not found
            return None

        # Extract the substring between the second occurrence of start_string and end_string
        extracted_text = text[start_index + len(start_string):end_index].strip()

        return extracted_text

def retrieve_general_information(url):
    '''Retrieves the general information section from the IMSLP page of a musical score until Instrumentation.'''
    # retrieve the general information section
    parsed_text = extract_text_between_strings_second(get_page_text(url),'General Information','Instrumentation')
    return parsed_text

def extract_text_between_strings_new(text, start_string, end_string):
    '''
    Extracts the substring between start_string and end_string in text.
    '''
    start_index = text.find(start_string)
    if start_index == -1:  # If start_string is not found
        return None

    # Find the occurrence of the end_string
    end_index = text.find(end_string, start_index)
    if end_index == -1:  # If end_string is not found
        return None

    # Extract the substring between the second occurrence of start_string and end_string
    extracted_text = text[start_index + len(start_string):end_index].strip()

    return extracted_text

In [ ]:
# find general information section

attribute_list = ['Work Title', 'Alt\nernative\n.\nTitle', 'Name Translations', 'Name Aliases', 'Authorities', 'Composer',
                        'Opus/Catalogue Number', 'I-Catalogue Number', 'Key', 'Movements/Sections', 'Year/Date of Composition', 
                        'First Pub\nlication', 'Librettist', 'Language', 'Copyright Information','Dedication', 'Average Duration', 'Composer Time Period', "Piece Style"]
list_of_deletable_headings = ['Opus/Catalogue Number','I-Catalogue Number','Movements/Sections','Year/Date of Composition','First Pub\nlication','Average Duration','Composer Time Period']
bad_heading_pattern = r'^.*\n'

info = {}
for pdf_id in ids:
    parsed_text = retrieve_general_information(url)
    
    if parsed_text:

        info_dic = {}
        
        present_attributes = []
        # check which attributes are present in the parsed text
        for attribute in attribute_list:
            if parsed_text.find(attribute) != -1:
                present_attributes.append(attribute)
        
        # extract the text for each present attribute
        for attribute in attribute_list:
            if attribute in present_attributes:
                info_dic[attribute] = extract_text_between_strings_new(parsed_text, attribute, present_attributes[present_attributes.index(attribute) + 1])
                
        info_dic['Piece Style'] = parsed_text[parsed_text.find('Piece Style')+12:] # extract the piece style from the end of the general information section
        info_dic['Instrumentation'] = 'Piano'
        info_dic['url'] = url
        for heading in list_of_deletable_headings:
            if info_dic[heading]:
                info_dic[heading] = re.sub(bad_heading_pattern, '', info_dic[heading])
        info[pdf_id] = info_dic
        
    else:
        print("String not found or invalid input.")

In [ ]:
# create pickle file from the dictionary
with open('9_way_metadata.pkl', 'wb') as f:
    pickle.dump(info, f)